In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [7]:
df = pd.read_excel("C:/Users/demir/OneDrive/Masaüstü/kazıma/Hepsi_Emlak/HepsiEmlakWebScraping/Hepsi_Emlak.xlsx")
df.head()

,Şehir,İlçe,Fiyat,Metre_Kare,Konut Tipi,Oda + Salon Sayısı,Bulunduğu Kat,Bina Yaşı,Isınma Tipi,Kat Sayısı,Eşya Durumu,Banyo Sayısı,Kira Getirisi
0,Adana,Sarıçam,1.895.000 TL,175 m2,Daire,3 + 1,4. Kat,1 Yaşında,Kombi,13 Katlı,Eşyalı Değil,2.0,15.000 TL
1,Adana,Sarıçam,19.500.000 TL,628 m2,Villa,5 + 1,NaN,Sıfır Bina,Kombi,3 Katlı,Eşyalı Değil,3.0,1.000.000 TL
2,İstanbul,Beylikdüzü,3.150.000 TL,125 m2,Daire,2 + 1,2. Kat,3 Yaşında,Kombi,4 Katlı,Eşyalı Değil,1.0,30.000 TL
3,Ankara,Mamak,3.900.000 TL,260 m2,Daire,4 + 1,7. Kat,5 Yaşında,Kombi,7 Katlı,Eşyalı Değil,2.0,25.000 TL
4,Bursa,Osmangazi,3.750.000 TL,170 m2,Daire,4 + 1,2. Kat,5 Yaşında,Kombi,2 Katlı,Eşyalı Değil,1.0,NaN


In [8]:
df.shape

(19085, 13)

In [9]:
df.isnull().sum()

Şehir                     0
İlçe                      0
Fiyat                     0
Metre_Kare                0
Konut Tipi                0
Oda + Salon Sayısı        0
Bulunduğu Kat          2344
Bina Yaşı                 0
Isınma Tipi               2
Kat Sayısı                2
Eşya Durumu            1219
Banyo Sayısı            216
Kira Getirisi         10972
dtype: int64

In [10]:
df.drop_duplicates(inplace=True)
df.shape


(16029, 13)

In [11]:
df.columns = [
    "City", "District", "Price(TL)", "Field(m²)", "Housing Type", 
    "Number of Rooms + Living Room", "Floor Location", "Building Age", 
    "Heating Type", "Number of Floors", "Property Status", 
    "Number of Bathrooms", "Rental Income(TL)"
]


In [12]:
df["Rental Income(TL)"] = df["Rental Income(TL)"].str.replace(" TL", "", regex=False)
df["Rental Income(TL)"] = df["Rental Income(TL)"].str.replace(".", "", regex=False)
df["Rental Income(TL)"] = df["Rental Income(TL)"].str.replace(",", "", regex=False)
df.loc[df["Rental Income(TL)"].str.contains("GBP", na=False), "Rental Income(TL)"] = (
df["Rental Income(TL)"].str.replace(" GBP", "", regex=False).str.replace(".", "", regex=False).astype("Int64") * 44) #22 Oct
df["Rental Income(TL)"] = df["Rental Income(TL)"].astype("float64")



In [13]:
df["Rental Income(TL)"] = df["Rental Income(TL)"].fillna(df.groupby(['City', "District", "Field(m²)" , "Price(TL)"])['Rental Income(TL)'].transform("mean"))
df["Rental Income(TL)"] = df["Rental Income(TL)"].fillna(df.groupby(['City', "District", "Field(m²)"])['Rental Income(TL)'].transform("mean"))
df["Rental Income(TL)"] = df["Rental Income(TL)"].fillna(df.groupby(['City', "District"])['Rental Income(TL)'].transform("mean"))

In [14]:
df["Price(TL)"] = df["Price(TL)"].str.replace(" TL", "")
df["Price(TL)"] = df["Price(TL)"].str.replace(".", "")
df.loc[df["Price(TL)"].str.contains("GBP"), "Price(TL)"] = (
df["Price(TL)"].str.replace(" GBP", "").str.replace(".", "").astype("Int64") * 44) #22 Oct
df["Price(TL)"] = df["Price(TL)"].astype('int64')


df["Field(m²)"] = df["Field(m²)"].str.replace(" m2", "")
df["Field(m²)"] = df["Field(m²)"].replace("Kat Karşılığı :", "0")
df["Field(m²)"] = df["Field(m²)"].str.replace(".", "", regex=False)
df["Field(m²)"] = df["Field(m²)"].astype("int64")


df["Building Age"] = df["Building Age"].str.replace("Sıfır Bina", "0")
df["Building Age"] = df["Building Age"].str.replace(" Yaşında", "")
df["Building Age"] = df["Building Age"].astype("int64")


df["Number of Floors"] = df["Number of Floors"].str.replace(" Katlı", "", regex=False)


df[['Number of Rooms', 'Living_Rooms']] = df['Number of Rooms + Living Room'].str.split(r'\s*\+\s*', expand=True)
df.drop("Number of Rooms + Living Room", axis=1,inplace=True)
df['Number of Rooms'] = df['Number of Rooms'].astype("int64")
df['Living_Rooms'] = df['Living_Rooms'].astype("int64")
df = df.dropna(subset=['Number of Bathrooms'])
df["Number of Bathrooms"] = df["Number of Bathrooms"].astype('int64')


df.shape


(15846, 14)

In [15]:
df.isnull().sum()

City                      0
District                  0
Price(TL)                 0
Field(m²)                 0
Housing Type              0
Floor Location         1967
Building Age              0
Heating Type              2
Number of Floors          2
Property Status        1015
Number of Bathrooms       0
Rental Income(TL)       167
Number of Rooms           0
Living_Rooms              0
dtype: int64

In [16]:
df = df.dropna(how="any")

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12903 entries, 0 to 19083
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 12903 non-null  object 
 1   District             12903 non-null  object 
 2   Price(TL)            12903 non-null  int64  
 3   Field(m²)            12903 non-null  int64  
 4   Housing Type         12903 non-null  object 
 5   Floor Location       12903 non-null  object 
 6   Building Age         12903 non-null  int64  
 7   Heating Type         12903 non-null  object 
 8   Number of Floors     12903 non-null  object 
 9   Property Status      12903 non-null  object 
 10  Number of Bathrooms  12903 non-null  int64  
 11  Rental Income(TL)    12903 non-null  float64
 12  Number of Rooms      12903 non-null  int64  
 13  Living_Rooms         12903 non-null  int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 1.5+ MB


In [18]:
df["Rental Income(TL)"] = df["Rental Income(TL)"].astype("int64")
df["Number of Floors"] = df["Number of Floors"].astype('int64')

In [19]:
lbe = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
        df[col] = lbe.fit_transform(df[col])

In [20]:
df.head()

,City,District,Price(TL),Field(m²),Housing Type,Floor Location,Building Age,Heating Type,Number of Floors,Property Status,Number of Bathrooms,Rental Income(TL),Number of Rooms,Living_Rooms
0,0,213,1895000,175,1,15,1,9,13,1,2,15000,3,1
2,65,48,3150000,125,1,11,3,9,4,1,1,30000,2,1
3,5,166,3900000,260,1,18,5,9,7,1,2,25000,4,1
4,16,197,3750000,170,1,11,5,9,2,1,1,14822,4,1
5,9,78,3650000,110,1,11,15,7,4,1,1,25000,2,1


In [21]:
Q1 = df["Rental Income(TL)"].quantile(0.80)
Q3 = df["Rental Income(TL)"].quantile(0.90)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers1 = df[(df["Rental Income(TL)"] < lower) | (df["Rental Income(TL)"] > upper)]

df = df[(df["Rental Income(TL)"] >= lower) & (df["Rental Income(TL)"] <= upper)]

print(f"Q1 (1st quartile): {Q1}")
print(f"Q3 (3rd quartile): {Q3}")
print(f"IQR: {IQR}")
print(f"Lower bound: {lower}")
print(f"Upper bound: {upper}")
print(f"Number of outliers: {len(outliers1)}\n")


Q1 (1st quartile): 32500.0
Q3 (3rd quartile): 47333.200000000914
IQR: 14833.200000000914
Lower bound: 10250.19999999863
Upper bound: 69583.00000000228
Number of outliers: 1566



In [22]:
Q1 = df["Price(TL)"].quantile(0.79)
Q3 = df["Price(TL)"].quantile(0.90)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df["Price(TL)"] < lower_bound) | (df["Price(TL)"] > upper_bound)]

df_filtered = df[(df["Price(TL)"] >= lower_bound) & (df["Price(TL)"] <= upper_bound)]

print(f"Q1 (1st quartile): {Q1}")
print(f"Q3 (3rd quartile): {Q3}")
print(f"IQR: {IQR}")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")
print(f"Number of outliers: {len(outliers)}\n")


Q1 (1st quartile): 6350000.0
Q3 (3rd quartile): 9900000.0
IQR: 3550000.0
Lower bound: 1025000.0
Upper bound: 15225000.0
Number of outliers: 570



In [23]:
df.shape

(11337, 14)

In [24]:
df.to_excel("Hepsi_Emlak_Data_Cleaned.xlsx", index = False)